<a href="https://colab.research.google.com/github/NaikVedhas/BumbleBee/blob/main/DS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from skimage.color import rgb2gray
from skimage.feature import hog
from PIL import Image, UnidentifiedImageError

In [ ]:
def img_to_np(img_path):
    try:
        img = Image.open(img_path)
        img = img.convert("RGB") # ensure img is in RGB
        new_img = img.resize((64, 64))
        return np.array(new_img)

    except (IOError, UnidentifiedImageError):
        return None

In [ ]:
def row_of_features(img):
    # 3 channel img to single row
    color_features = img.flatten()

    gray_img = rgb2gray(img)

    hog_features = hog(gray_img, block_norm='L2-Hys', pixels_per_cell=(16, 16))

    # combine color & hog features into a single array
    row = np.hstack((color_features, hog_features))

    return row

In [ ]:
def gradient_descent(X, y, alpha, iterations):
    n, k = X.shape

    X = np.c_[np.ones((n, 1)), X] # Adds a col of ones at the beginning

    beta = np.zeros(k + 1) # initialize beta

    for i in range(iterations):
        y_pred = sigmoid(np.dot(X, beta))

        gradient = (1 / n) * np.dot(X.T, (y_pred - y))

        beta -= alpha * gradient # new beta values

    return beta

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def predict(X, beta):
    X = np.c_[np.ones((X.shape[0], 1)), X] # Adds a col of ones at the beginning

    probabilites = sigmoid(np.dot(X, beta))
    y_pred = (probabilites >= 0.5).astype(int)

    return y_pred

In [ ]:
!pip install importnb


In [ ]:
import os
import joblib
import importnb



In [ ]:
# load necessary models

ss = joblib.load("Models/standard_scaler.pkl")
pca = joblib.load("Models/pca.pkl")

In [ ]:
def predict_img(img_path, model_name):
    model_path = os.path.join("Models", f"{model_name}.pkl")
    model = joblib.load(model_path)

    np_img = img_to_np(img_path)
    features = row_of_features(np_img)
    features = features.reshape(1, -1)

    features_ss = ss.transform(features)
    features_pca = pca.transform(features_ss)

    if (model_name == "logistic_regression"):
        beta = model
        prediction = predict(features_pca, beta)

    else:
        prediction = model.predict(features_pca)

    if prediction == 0:
        print("Genus: Apis")

    else:
        print("Genus: Bombus")

In [ ]:
img_path = r"/content/images (1).jpg" # Replace with your image path

In [ ]:
print("Original image: apis")
predict_img(img_path, "logistic_regression") # - sahi
predict_img(img_path, "knn")
predict_img(img_path, "naive_bayes")
predict_img(img_path, "svm")  #svm
predict_img(img_path, "decision_tree") #- sahi
predict_img(img_path, "random_forest")

Original image: apis
Genus: Bombus
Genus: Bombus
Genus: Apis
Genus: Bombus
Genus: Apis
Genus: Bombus
